# Import Required Libraries

In [1]:
# KERAS
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
# from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras import optimizers


# Useful Imports
from glob import glob
import re
import os
import numpy as np
import pickle

Using TensorFlow backend.


# Model MK1  ( Checking Pre-Treined VGG16)

In [2]:
# model = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling=None)
# # weights='imagenet'
# # input_shape=(224,224,3)
# # classes=4

In [3]:
from keras.models import load_model
model = load_model('FifthModel.h5')

img = load_img('/home/rodrigo/Batch/0/pngs/CT11.png')
# convert the image pixels to a numpy array
image = img_to_array(img)
print(image.shape)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
print(image.shape)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
(224, 224, 3)
(1, 224, 224, 3)


In [4]:
# prepare the image for the VGG model
image = preprocess_input(image)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# # predict the probability across all output classes
yhat = model.predict(image)
print(yhat)

[[1.]]


In [ ]:
# convert the probabilities to class labels
# label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
# label = label[0][0]
# print the classification
# print('%s (%.2f%%)' % (label[1], label[2]*100))

# Model MK2 ( Remove Classification Layers from VGG16)

In [ ]:
# #add our own fully connected layers for the final classification
# base_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling=None)

In [ ]:
# x = base_model.output
# #flattentit
# x = Flatten()(x)
# # let's add a fully-connected layer
# x = Dense(4096, activation='relu')(x)
# #another fully-connected layer
# x = Dense(4096, activation='relu')(x)
# # and a logistic layer -- let's say we have 200 classes
# predictions = Dense(1, kernel_initializer='normal', activation='sigmoid')(x)

# # this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)
# #now we can start to fine-tune the model
# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional InceptionV3 layers
# for layer in base_model.layers:
#     layer.trainable = False
    
# # compile the model (should be done *after* setting layers to non-trainable)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prepare dataset

In [ ]:
# import dataset
data_dir = '/home/rodrigo/processed-dataset/'
num_slices_per_subject = 28
# local = "numpy"
local = "windowed"

# create list of IDs from all slices
all_ids = set()
all_slices = glob(data_dir + "**/" + local + "/*.npy")
for item in all_slices:
    subj_match = re.match(data_dir + '([0-9]+)/' + local + '/scan_([0-9]+)_slice_([0-9]+).npy', item)
    exam_id = subj_match.group(1)
    all_ids = all_ids.union([exam_id])
print("total scans: " + str(len(all_ids)))
print("total slices: " + str(len(all_slices)))
print(sorted(all_ids))

# 20 scans for validation
all_ids = list(all_ids)
index = int(np.floor(len(all_ids)/10*9.53))
validation_ids = all_ids[index:]
train_and_test_ids = all_ids[0:index]
print("for validation: " + str(len(validation_ids)))
print("for train/test: " + str(len(train_and_test_ids)))

# list with slices filenames: [scan_0_slice0, scan_0_slice1, scan_1_slice0....]
# filenames for slices and labels are equal
all_slices_ids = list()
for subj_id in train_and_test_ids:
    for slice_fn in os.listdir(data_dir + subj_id + "/" + local + "/"):
        all_slices_ids.append(slice_fn)

print("training and test slices: " + str(len(all_slices_ids)))

# divide list into train and test
percentage_to_train = 0.8
# cutoff_index: last slice index from last scan based of percentage_to_train
cutoff_index = int(np.floor(len(train_and_test_ids) * percentage_to_train)) * num_slices_per_subject
training_ids = all_slices_ids[0:cutoff_index]
validation_ids  = all_slices_ids[cutoff_index:]
print("slices for training: " + str(cutoff_index))
print("slices for test: " + str(len(all_slices_ids) - cutoff_index))

total scans: 410
total slices: 11344
['0', '1', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '113', '116', '117', '119', '12', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '132', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '152', '154', '155', '156', '159', '160', '162', '164', '165', '166', '167', '168', '17', '171', '172', '173', '174', '175', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '2', '20', '200', '202', '205', '207', '208', '21', '211', '212', '213', '214', '215', '216', '217', '218', '219', '22', '220', '221', '222', '223', '225', '226', '227', '228', '229', '23', '231', '232', '233', '234', '235', '236', '237', '238', '239', '24', '241', '242', '243', '244', '245', '246', '248', '249', '25', '250', '251', '252', '

# Model MK3

In [ ]:
from CQ500DataGenerator import DataGenerator
print(training_ids[0])
print(validation_ids[0])
training_generator = DataGenerator(training_ids)
validation_generator = DataGenerator(validation_ids)
print(validation_generator[0])
print(training_generator[0])

scan_148_slice_31.npy
scan_86_slice_6.npy
(array([[[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0., 85.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0., 85.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0., 85.,  0.]],

        ...,

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0., 85.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0., 85.,  0.]],

        [[ 0., 85.,  0.],
         [ 0., 85.,  0.],
         [ 0., 85.,  0.],
     

In [ ]:
# create the base pre-trained model
base_model = VGG16(include_top=False, weights='imagenet', 
                    input_tensor=None, input_shape=(224, 224, 3), pooling=None)

#now we can start to fine-tune the model
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
print("--------------------------------------")
print(x.shape)
print("--------------------------------------")
#flatten it
x = Flatten()(x)
x = Dropout(0.2)(x)
# let's add a fully-connected layer
x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)
#x = LeakyReLU(alpha=.01)(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.2)(x)

#another fully-connected layer
#x = Dense(200, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

print(model.summary())

my_optimizer=optimizers.Adam(lr=0.00001)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='binary_crossentropy', optimizer=my_optimizer, metrics=['accuracy'])
history = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=5, use_multiprocessing=False)
    
model.save('SixthModel.h5')

with open('SixthModel_trainHistoryDict', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)


--------------------------------------
(?, 7, 7, 512)
--------------------------------------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________

In [ ]:
str = '/home/rodrigo/processed-dataset/467/numpy/scan_467_slice_5.npy'

a = np.load(str)
print(a.shape)

# Save Merged Label File

In [ ]:
# path = '/home/rodrigo/Projects/TCC/'
# file = 'labels2.csv'
# labels = np.loadtxt(open(path + file, "rb"), delimiter=",", skiprows=1)
# exams = labels[:, 0]
# ich = [ int(x/2) for x in np.sum(labels[:,1:4], axis=1) ]
# iph = [ int(x/2) for x in np.sum(labels[:,4:7], axis=1)]
# ivh = [ int(x/2) for x in np.sum(labels[:,7:10], axis=1)]
# sdh = [ int(x/2) for x in np.sum(labels[:,10:13], axis=1)]
# edh = [ int(x/2) for x in np.sum(labels[:,13:16], axis=1)]
# sah = [ int(x/2) for x in np.sum(labels[:,16:19], axis=1)]
# result = np.transpose(np.matrix([exams, ich, iph, ivh, sdh, edh, sah]))
# np.savetxt(path + 'labels-merged.csv', result, delimiter=",")